## Import the required Libraries
- `numpy as np`: Numerical computing library for array operations and mathematical functions.
- `pandas as pd`: Data manipulation and analysis library, used for loading and handling datasets (e.g., CSV files).
- `tensorflow as tf`: Machine learning framework for building and training neural networks, including the Keras API.
- `tensorflow.keras.models`: Keras models module for creating sequential models and loading saved models.
- `tensorflow.keras.layers`: Keras layers module for defining neural network layers such as Conv2D, MaxPooling2D, Flatten, Dense, Dropout, and BatchNormalization.
- `tensorflow.keras.utils`: Keras utilities module, including to_categorical for converting labels to one-hot encoded format.
- `sklearn.model_selection`: Scikit-learn module for splitting data into training and testing sets.
- `cv2`: OpenCV library for image processing, such as loading and preprocessing images.
- `matplotlib.pyplot as plt`: Matplotlib library for plotting and visualizing data.
- `webbrowser`: Python module for opening URLs in a web browser.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import webbrowser

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Preprocessing

This section outlines the preprocessing function for an emotion recognition model, handling missing data and preparing image and label data.

## Emotion Classes
- `emotion_classes`: List of emotion categories: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral.

## Preprocessing Function
- **Important Note**: Ensure all pixel data is properly formatted as strings and can be reshaped to 48x48x1 arrays to avoid runtime errors.
- `preprocess(data)`: Function to preprocess the dataset.
  - Checks for missing values and drops rows if found.
  - Extracts and normalizes pixel data into 48x48x1 images.
  - One-hot encodes emotion labels into 7 classes.

In [3]:
emotion_classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def preprocess(data):
    # Check for missing values
    if data.isnull().values.any():
        print("Missing values found. Dropping rows with missing data...")
        data = data.dropna().reset_index(drop=True)
    else:
        print("No missing values found.")

    # Extract and normalize pixel data
    pixels = data['pixels'].tolist()
    images = np.array([
        np.fromstring(pixel, sep=' ').reshape(48, 48, 1) for pixel in pixels
    ])
    images = images / 255.0  # Normalize

    # One-hot encode emotion labels
    labels = to_categorical(data['emotion'].values, num_classes=7)

    return images, labels



## Data Loading and Splitting
- `data`: Loaded from `fer2013.csv` using pandas.
- `train_data`: Subset where `Usage` is 'Training'.
- `val_data`: Subset where `Usage` is 'PublicTest'.
- `test_data`: Subset where `Usage` is 'PrivateTest'.

## Preprocessing
- `X_train, y_train`: Preprocessed training data and labels.
- `X_val, y_val`: Preprocessed validation data and labels.
- `X_test, y_test`: Preprocessed test data and labels.

In [5]:
data = pd.read_csv(r"C:\Users\SWAROOP\Desktop\fer2013.csv")

train_data = data[data['Usage'] == 'Training']
val_data = data[data['Usage'] == 'PublicTest']
test_data = data[data['Usage'] == 'PrivateTest']

X_train, y_train = preprocess(train_data)
X_val, y_val = preprocess(val_data)
X_test, y_test = preprocess(test_data)


No missing values found.
No missing values found.
No missing values found.


# Building the Emotion Recognition Model

This section defines the architecture and compilation of a Convolutional Neural Network (CNN) for emotion recognition.

## Model Architecture
- `build_model()`: Function to create the CNN model.

### Input Layer
- **Layer 1**: Conv2D with 64 filters, 3x3 kernel, ReLU activation, input shape (48, 48, 1).

### Hidden Layers
- **Layer 2**: BatchNormalization for stabilizing training.
- **Layer 3**: MaxPooling2D with 2x2 pool size.
- **Layer 4**: Conv2D with 128 filters, 3x3 kernel, ReLU activation.
- **Layer 5**: BatchNormalization.
- **Layer 6**: MaxPooling2D with 2x2 pool size.
- **Layer 7**: Conv2D with 256 filters, 3x3 kernel, ReLU activation.
- **Layer 8**: BatchNormalization.
- **Layer 9**: MaxPooling2D with 2x2 pool size.
- **Layer 10**: Flatten to convert 2D to 1D.
- **Layer 11**: Dense with 512 units, ReLU activation.
- **Layer 12**: Dropout with 0.5 rate to prevent overfitting.

### Output Layer
- **Layer 13**: Dense with 7 units, softmax activation for 7 emotion classes.

## Model Compilation
- `model.compile()`: Configures the model with:
  - Optimizer: 'adam'.
  - Loss function: 'categorical_crossentropy'.
  - Metrics: 'accuracy'.

In [7]:
def build_model():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    return model

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\SWAROOP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training and Evaluating

This section covers the training, evaluation, and saving of the emotion recognition model.

## Training the Model
- `model.fit()`: Trains the model with:
  - Training data: `X_train`, `y_train`.
  - Epochs: 10.
  - Batch size: 64.
  - Validation data: `X_val`, `y_val`.

## Evaluating the Model
- `model.evaluate()`: Assesses the model on test data `X_test`, `y_test`.
  - Returns test loss and test accuracy.
  - Prints test accuracy with 2 decimal places.

## Saving the Model
- `model.save()`: Saves the trained model to 'emotion_recognition_model.h5'.

In [9]:
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

model.save('emotion_recognition_model.h5')



Epoch 1/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 126s 267ms/step - accuracy: 0.2795 - loss: 2.3486 - val_accuracy: 0.3031 - val_loss: 1.7117
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 138s 259ms/step - accuracy: 0.4240 - loss: 1.4770 - val_accuracy: 0.3943 - val_loss: 1.5804
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 117s 260ms/step - accuracy: 0.4769 - loss: 1.3684 - val_accuracy: 0.2435 - val_loss: 1.9236
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 116s 258ms/step - accuracy: 0.5212 - loss: 1.2583 - val_accuracy: 0.4650 - val_loss: 1.3762
Epoch 5/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 116s 257ms/step - accuracy: 0.5463 - loss: 1.1899 - val_accuracy: 0.5300 - val_loss: 1.2476
Epoch 6/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 118s 262ms/step - accuracy: 0.5754 - loss: 1.1232 - val_accuracy: 0.4999 - val_loss: 1.3084
Epoch 7/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 117s 260ms/step - accuracy: 0.6127 - loss: 1.0338 - val_accuracy: 0.5439 - val_loss: 1.2241
Epoch 8/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 118s 262ms/step - accuracy: 0.6443 -

# Predicting Emotion from Image

This section defines a function to predict emotions from a given image using the trained model.

## Prediction Function
- `predict_emotion_from_image(img_path, model)`: Predicts emotion from an image.
  - Loads the image using `cv2.imread()`.
  - Converts the image to grayscale with `cv2.cvtColor()`.
  - Resizes the image to 48x48 with `cv2.resize()`.
  - Reshapes and normalizes the image data.
  - Uses `model.predict()` to get the prediction.
  - Determines the emotion using `emotion_classes` and `np.argmax()`.
  - Displays the image with the predicted emotion using `matplotlib`.
  - Returns the predicted emotion.

In [11]:
def predict_emotion_from_image(img_path, model):
    img = cv2.imread(img_path)
    if img is None:
        print("Error: Image not found.")
        return None
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (48, 48))
    reshaped = resized.reshape(1, 48, 48, 1).astype('float32') / 255.0
    pred = model.predict(reshaped)
    emotion = emotion_classes[np.argmax(pred)]

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.title(f"Predicted Emotion: {emotion}")
    plt.axis('off')
    plt.show()

    return emotion


# Predicting Emotion from Video Frame

This section defines a function to predict emotions from a video frame using the trained model with face detection.

## Prediction Function
- `predict_emotion_from_frame(frame, model)`: Predicts emotion from a video frame.
  - Converts the frame to grayscale with `cv2.cvtColor()`.
  - Loads the Haar Cascade Classifier for face detection.
  - Detects faces using `detectMultiScale()`.
  - For each detected face:
    - Extracts the region of interest (ROI).
    - Resizes the ROI to 48x48 with `cv2.resize()`.
    - Reshapes and normalizes the image data.
    - Uses `model.predict()` to get the prediction.
    - Determines the emotion using `emotion_classes` and `np.argmax()`.
    - Draws a rectangle around the face and adds the emotion label with `cv2.rectangle()` and `cv2.putText()`.
    - Returns the first detected emotion.
  - Returns `None` if no faces are detected.

In [13]:
def predict_emotion_from_frame(frame, model):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        resized = cv2.resize(roi, (48, 48))
        reshaped = resized.reshape(1, 48, 48, 1).astype('float32') / 255.0
        pred = model.predict(reshaped)
        emotion = emotion_classes[np.argmax(pred)]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)

        return emotion
    return None


# Emotion-Based Recommendations

This section defines a dictionary of recommendations and a function to provide tailored suggestions based on predicted emotions.

## Recommendations Dictionary
- `recommendations`: A dictionary mapping emotions to:
  - `quote`: Inspirational quote.
  - `song`: Suggested song with a YouTube URL.
  - `url`: Link to the song on YouTube.
  - `activity`: Suggested activity to enhance mood.

### Emotion Categories
- **Angry**: Quote by Ralph Waldo Emerson, song "Calm Down" by Taylor Swift, activity: deep breathing or walking.
- **Disgust**: Quote about letting go, song "Clean" by Taylor Swift, activity: declutter or meditate.
- **Fear**: Quote by Eleanor Roosevelt, song "Brave" by Sara Bareilles, activity: journal or talk to a friend.
- **Happy**: Quote by Dalai Lama, song "Happy" by Pharrell Williams, activity: celebrate or share joy.
- **Sad**: Quote by Robert H. Schuller, song "Fix You" by Coldplay, activity: watch a movie or call a loved one.
- **Surprise**: Quote about embracing life, song "Surprise Yourself" by Jack Garratt, activity: try something new.
- **Neutral**: Quote about doing nothing, song "Let It Be" by The Beatles, activity: take a break or enjoy tea.

## Recommendation Function
- `recommend_based_on_emotion(emotion)`: Provides recommendations based on the given emotion.
  - Retrieves recommendation from the dictionary.
  - Prints quote, song, and activity.
  - Opens the song URL in a web browser.
  - Prints "No recommendation available" if emotion is not found.

In [17]:
recommendations = {
    'Angry': {
        'quote': "For every minute you remain angry, you give up sixty seconds of peace of mind. – Ralph Waldo Emerson",
        'songs': {
            'English': [
                {"title": "Calm Down – Taylor Swift", "url": "https://www.youtube.com/watch?v=nfWlot6h_JM"},
                {"title": "Let It Go – Idina Menzel", "url": "https://www.youtube.com/watch?v=L0MK7qz13bU"}
            ],
            'Telugu': [
                {"title": "Pranama – Orange", "url": "https://www.youtube.com/watch?v=abVQHq2WuhU"},
                {"title": "Manasa – Munna", "url": "https://www.youtube.com/watch?v=kCJVR8X1NoY"}
            ]
        },
        'activity': "Try deep breathing or go for a walk."
    },
    'Disgust': {
        'quote': "Let go of what doesn't serve you.",
        'songs': {
            'English': [
                {"title": "Clean – Taylor Swift", "url": "https://www.youtube.com/watch?v=WA4iX5D9Z64"},
                {"title": "Demons – Imagine Dragons", "url": "https://www.youtube.com/watch?v=mWRsgZuwf_8"}
            ],
            'Telugu': [
                {"title": "Chal Chalo Chalo – Happy Days", "url": "https://www.youtube.com/watch?v=4HDtPo4fK-0"},
                {"title": "Em Cheppanu – Oohalu Gusagusalade", "url": "https://www.youtube.com/watch?v=mnZc9F9NjF8"}
            ]
        },
        'activity': "Do a quick declutter or meditate."
    },
    'Fear': {
        'quote': "Do one thing every day that scares you. – Eleanor Roosevelt",
        'songs': {
            'English': [
                {"title": "Brave – Sara Bareilles", "url": "https://www.youtube.com/watch?v=QUQsqBqxoR4"},
                {"title": "Fight Song – Rachel Platten", "url": "https://www.youtube.com/watch?v=xo1VInw-SKc"}
            ],
            'Telugu': [
                {"title": "Chakkani Bike Undi – Ee Nagaraniki Emaindi", "url": "https://www.youtube.com/watch?v=izkKeHz7uRc"},
                {"title": "Nuvvu Leka Nenu Lenu – Gangotri", "url": "https://www.youtube.com/watch?v=IJCG8sDLpEo"}
            ]
        },
        'activity': "Journal your thoughts or talk to a friend."
    },
    'Happy': {
        'quote': "Happiness is not something ready-made. It comes from your own actions. – Dalai Lama",
        'songs': {
            'English': [
                {"title": "Happy – Pharrell Williams", "url": "https://www.youtube.com/watch?v=ZbZSe6N_BXs"},
                {"title": "Best Day of My Life – American Authors", "url": "https://www.youtube.com/watch?v=Y66j_BUCBMY"}
            ],
            'Telugu': [
                {"title": "Oopiri Aaguthunnadey – Tholi Prema", "url": "https://www.youtube.com/watch?v=3Ih04UotIBk"},
                {"title": "Feel My Love – Arya", "url": "https://www.youtube.com/watch?v=jH3Md3eAFw0"}
            ]
        },
        'activity': "Celebrate the moment or share your joy!"
    },
    'Sad': {
        'quote': "Tough times never last, but tough people do. – Robert H. Schuller",
        'songs': {
            'English': [
                {"title": "Fix You – Coldplay", "url": "https://www.youtube.com/watch?v=k4V3Mo61fJM"},
                {"title": "Someone Like You – Adele", "url": "https://www.youtube.com/watch?v=hLQl3WQQoQ0"}
            ],
            'Telugu': [
                {"title": "Kanulanu Thaake – Geethanjali", "url": "https://www.youtube.com/watch?v=ZJP6bBoTbko"},
                {"title": "Evare – Premam", "url": "https://www.youtube.com/watch?v=nA0zHIVKwlA"}
            ]
        },
        'activity': "Watch a feel-good movie or call a loved one."
    },
    'Surprise': {
        'quote': "Life is full of surprises. Embrace them!",
        'songs': {
            'English': [
                {"title": "Surprise Yourself – Jack Garratt", "url": "https://www.youtube.com/watch?v=5gHq6aY1bnY"},
                {"title": "Pocketful of Sunshine – Natasha Bedingfield", "url": "https://www.youtube.com/watch?v=yjuW7Hw4K3s"}
            ],
            'Telugu': [
                {"title": "Yemito – Ala Modalaindi", "url": "https://www.youtube.com/watch?v=g5cCJfvAc88"},
                {"title": "Pilla – Gabbar Singh", "url": "https://www.youtube.com/watch?v=8XzMzpP9R8Q"}
            ]
        },
        'activity': "Try something new or spontaneous!"
    },
    'Neutral': {
        'quote': "Sometimes doing nothing is doing something.",
        'songs': {
            'English': [
                {"title": "Let It Be – The Beatles", "url": "https://www.youtube.com/watch?v=QDYfEBY9NM4"},
                {"title": "The Lazy Song – Bruno Mars", "url": "https://www.youtube.com/watch?v=fLexgOxsZu0"}
            ],
            'Telugu': [
                {"title": "Aaradugula Bullet – Attarintiki Daredi", "url": "https://www.youtube.com/watch?v=yWhd3Jp_pKk"},
                {"title": "Ee Manase – Tholi Prema", "url": "https://www.youtube.com/watch?v=ffUJv1BoqZc"}
            ]
        },
        'activity': "Take a break or enjoy a cup of tea."
    }
}

import webbrowser

def recommend_based_on_emotion(emotion):
    rec = recommendations.get(emotion)
    if rec:
        print(f"\n🎧 Recommendation for {emotion}:")
        print(f"💬 Quote: {rec['quote']}")
        print(f"🎯 Activity: {rec['activity']}")

        print("\n🎵 English Playlist:")
        for idx, song in enumerate(rec['songs']['English'], 1):
            print(f"{idx}. {song['title']}")

        print("\n🎶 Telugu Playlist:")
        for idx, song in enumerate(rec['songs']['Telugu'], 1):
            print(f"{idx}. {song['title']}")

        # Optional: Let user choose a song to play
        lang = input("\nChoose playlist to open (English/Telugu): ").capitalize()
        if lang in rec['songs']:
            print(f"\nOpening first {lang} song: {rec['songs'][lang][0]['title']}")
            webbrowser.open(rec['songs'][lang][0]['url'])
        else:
            print("Invalid choice. No song played.")
    else:
        print("No recommendation available.")


# Emotion Detection Mode Selection

This section implements a script to choose between image or live video input for emotion detection and provide recommendations.

## Mode Selection and Execution
- `input_mode`: User input to select mode ('live' or 'image').
- `model`: Loads the trained model from 'emotion_recognition_model.h5'.

### Image Mode
- If `input_mode` is 'image':
  - Prompts for an image path.
  - Calls `predict_emotion_from_image()` to predict emotion.
  - If successful, calls `recommend_based_on_emotion()` with the result.

### Live Mode
- If `input_mode` is 'live':
  - Initializes video capture with `cv2.VideoCapture(0)`.
  - Continuously reads frames and predicts emotions with `predict_emotion_from_frame()`.
  - Displays the frame with detected emotions.
  - Exits on 'q' key press, releases capture, and closes windows.
  - If an emotion is detected, calls `recommend_based_on_emotion()`.

### Invalid Mode
- Prints "Invalid mode selected." for unrecognized input.

In [ ]:
input_mode = input("Enter input mode (live/image): ").lower()
model = load_model('emotion_recognition_model.h5')

if input_mode == "image":
    path = input("Enter image path: ")
    result = predict_emotion_from_image(path, model)
    if result:
        recommend_based_on_emotion(result)

elif input_mode == "live":
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        result = predict_emotion_from_frame(frame, model)
        cv2.imshow('Live Emotion Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    if result:
        recommend_based_on_emotion(result)

else:
    print("Invalid mode selected.")
